### Student Information
Name:黃子軒

Student ID:113062614

GitHub ID:Brady29655751

Kaggle name:Brady Huang

Kaggle private scoreboard snapshot:![](./img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__. 

In [2]:
### Begin Assignment Here

## Instruction 2
#### Participate in Kaggle Competition.

I implemented my work in Kaggle notebook. Below is part of my work in Kaggle competition. <br/>
The first part is provided by Kaggle originally to check the input files.

In [ ]:
# These are provided by kaggle and check the input files.
import numpy as np
import pandas as pd
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Then, include the needed library.

In [ ]:
# import library
import json
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

Here I've tried some data cleaning techniques, such as transform emoji into text and drop duplicates, but the result improves only a little. <br/>
For public leaderboard, cleaning improves from 0.31545 to 0.32339. <br/>
For private leaderboard, cleaning improves from 0.31230 to 0.31909.

In [ ]:
# This part is code without data cleaning.
data = []
with open('/kaggle/input/dm-2024-isa-5810-lab-2-homework/tweets_DM.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))

In [ ]:
# This part is some code with data cleaning.
import emoji
import pandas as pd

# This is emoji dict that replace specific emoji to text.
# I don't include all emoji here since it's only for showing my work.
emoji_dict = {
    '😂': '[joy]',
    '❤️': '[love]',
    '😭': '[cry]',
    '🙏': '[pray]',
    '😘': '[kiss]',
    '🔥': '[fire]',
    '🤔': '[think]',
    '😁': '[happy]',
    '👍': '[great]',
}

# This is the cleaning function.
def clean_emoji(text):
    for emj, emj_text in emoji_dict.items():
        text = text.replace(emj, emj_text)
    text = emoji.replace_emoji(text, replace='')
    return text.strip()

Below shows <b>part of</b> my preprocessing and feature extraction code. <br/>
Originally, I use 20% of data for training, then 30%, and finally 50%. <br/>
I don't use 100% data for training because the model I used cannot be accelerated by GPU. <br/>

In [ ]:
# Identify train and test by given identification.
train_data = df[df['identification'] == 'train']
test_data = df[df['identification'] == 'test']

# Use part of data to train the model. This percentage is changed during experiments.
train_data_sample = train_data.sample(frac=0.5, random_state=97)

# Get X and y for training
y_train_data = train_data_sample['emotion']
X_train_data = train_data_sample.drop(['tweet_id', 'emotion', 'identification', 'hashtags'], axis=1)

# This train test split is used for splitting training and validation.
# Here uses stratify to make sure the distribution is roughly the same as y.
X_train, X_test, y_train, y_test = train_test_split(
    X_train_data, y_train_data, test_size=0.2, random_state=97, stratify=y_train_data
)

#### Feature Extraction
# Use very basic TF-IDF. The max features is changed during experiments.
# TF-IDF is fit with train and not test.
tfidf = TfidfVectorizer(max_features=3000)  
X = tfidf.fit_transform(X_train['text']).toarray()
X_test = tfidf.transform(X_test['text'])

Below shows my Label Encoder and Random Forest Model. <br/>
This takes lots of time and memory when the TFIDF features grow.

In [ ]:
# Label Encoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Random Forest
clf = RandomForestClassifier()
clf.fit(X, y)
y_pred = clf.predict(X_test)

# Check Accuracy
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

# Go for test dataset
# This one is the testing data from leaderboard (identification = "test")
X_test_data = test_data.drop(['tweet_id', 'identification', 'hashtags'], axis=1)
X_test_data = tfidf.transform(X_test_data['text']).toarray()

y_test_pred = clf.predict(X_test_data)

# Inverse to original label and generate submissions.
y_pred_labels = label_encoder.inverse_transform(y_test_pred)

submission = pd.DataFrame({
    'id': test_data['tweet_id'],
    'emotion': y_pred_labels
}, index=False)


The above is the explanation for my code. <br/>
My experiment results are concluded in my report.

## Instruction 3

Report of my work is included in the pdf file in github repository.

[report](./report.pdf)